In [29]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
# from keras.preprocessing.image import load_img
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
import glob
import numpy as np
import pandas as pd

In [30]:
import tensorflow as tf
print(tf.__version__)

2.9.2


In [31]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
from tensorflow.keras.utils import load_img

In [33]:
# from keras.preprocessing.image import load_img
!pip version tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

ERROR: unknown command "version"


In [34]:
imagenet_classes = pd.read_json("imagenet_class_index.json")
imagenet_classes = imagenet_classes.T[[1]]

imagenet_classes.columns = ["Class"]

imagenet_classes

,Class
0,tench
1,goldfish
2,great_white_shark
3,tiger_shark
4,hammerhead
...,...
995,earthstar
996,hen-of-the-woods
997,bolete
998,ear


In [35]:
pre_trained_model = VGG16(weights="imagenet")

In [36]:
pre_trained_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### Image Classification - Predictions

In [37]:
image_for_prediction = load_img("hammerhead.jpg",
                               target_size=(224,224))

In [38]:
image_for_prediction = np.array(image_for_prediction)
image_for_prediction = preprocess_input(image_for_prediction)
image_for_prediction = image_for_prediction.reshape(1,224,224,3)

In [39]:
image_for_prediction.shape

(1, 224, 224, 3)

In [40]:
preds = pre_trained_model.predict(image_for_prediction)

1/1 [==============================] - 1s 682ms/step


In [41]:
imagenet_classes["preds"] = preds.flatten()

In [42]:
imagenet_classes.sort_values(by="preds",ascending=False)

,Class,preds
4,hammerhead,9.830524e-01
3,tiger_shark,9.417460e-03
2,great_white_shark,2.005062e-03
327,starfish,1.990423e-03
5,electric_ray,1.846810e-03
...,...,...
439,bearskin,7.261936e-10
573,go-kart,7.003905e-10
592,hard_disc,6.627712e-10
817,sports_car,4.307341e-10


### Feature Extraction

In [43]:
pre_trained_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [44]:
new_model = Sequential()

In [45]:
for layer in pre_trained_model.layers[:-1]:
    new_model.add(layer)

In [46]:
new_model.predict(image_for_prediction)

1/1 [==============================] - 1s 695ms/step


array([[0.       , 0.       , 2.4572763, ..., 0.       , 1.2113194,
        1.8231306]], dtype=float32)

### Finetuning

Use VGG16 pretrained model as baseline and train a binary class classification problem to classify planes from cars

In [47]:
new_model = Sequential()
for layer in pre_trained_model.layers[:-1]:
    layer.trainable = True # Will help backpropagate
    new_model.add(layer)

new_model.add(Dense(1,"sigmoid"))

In [48]:
new_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [49]:
cars = "/content/Cars/*"
planes = "/content/Planes/*"

In [50]:
x_train = list()
for i in glob.glob(cars):
    input_img = load_img(i, target_size=(224,224))
    input_img = np.array(input_img)
    x_train.append(input_img)
for i in glob.glob(planes):
    input_img = load_img(i, target_size=(224,224))
    input_img = np.array(input_img)
    x_train.append(input_img)

In [51]:
y_train = [0]*50 + [1]*50 # 0 - Car, 1 - Plane

In [52]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [53]:
print(x_train.shape)
print(y_train.shape)

(100, 224, 224, 3)
(100,)


In [54]:
new_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [55]:
new_model.fit(x=x_train, y=y_train, epochs=20, validation_split=0.2)

Epoch 1/20
3/3 [==============================] - 157s 46s/step - loss: 3742.4739 - accuracy: 0.6250 - val_loss: 1.1500e-26 - val_accuracy: 1.0000
Epoch 2/20
3/3 [==============================] - 147s 46s/step - loss: 25.9425 - accuracy: 0.5250 - val_loss: 0.0967 - val_accuracy: 1.0000
Epoch 3/20
3/3 [==============================] - 147s 46s/step - loss: 2.2304 - accuracy: 0.5125 - val_loss: 0.5115 - val_accuracy: 0.9000
Epoch 4/20
3/3 [==============================] - 146s 46s/step - loss: 11.7056 - accuracy: 0.5375 - val_loss: 13.1733 - val_accuracy: 0.0000e+00
Epoch 5/20
3/3 [==============================] - 146s 46s/step - loss: 3.3321 - accuracy: 0.6250 - val_loss: 0.9796 - val_accuracy: 0.1000
Epoch 6/20
3/3 [==============================] - 146s 46s/step - loss: 3.6170 - accuracy: 0.4375 - val_loss: 0.0769 - val_accuracy: 1.0000
Epoch 7/20
3/3 [==============================] - 146s 46s/step - loss: 1.2467 - accuracy: 0.3750 - val_loss: 1.3960 - val_accuracy: 0.0000e+00
Ep

In [63]:
new_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [66]:
image_for_pred = load_img("plane.jpg",
                               target_size=(224,224))

In [65]:
image_for_prediction = np.array(image_for_prediction)
image_for_prediction = preprocess_input(image_for_prediction)
image_for_prediction = image_for_prediction.reshape(1,224,224,3)

In [59]:
image_for_prediction.shape

(1, 224, 224, 3)

In [60]:
preds = new_model.predict(image_for_prediction)

1/1 [==============================] - 1s 666ms/step


In [61]:
preds.flatten()

array([0.37936562], dtype=float32)